In [108]:
import vectorbt as vbt
import numpy as np
import numpy.ma as ma
import pandas as pd
import inspect
import datetime
import talib
from IPython.display import display, HTML


vbt.settings.data['alpaca']['key_id'] = 'PK2XNKDSJH4PVPCYDK8E'
vbt.settings.data['alpaca']['secret_key'] = 'f5cD1kbR3p5RRfgvNgDiQ09qRpi4LCYmPZprl3KM'

In [109]:
symbol = 'tqqq'
data = vbt.AlpacaData.download(symbol,
                               start='1990-3-05',
                               end='2023-3-16',
                               limit=2000000,
                               timeframe='1d').get()

inspect.signature(data.vbt.ohlcv.plot)
data.vbt.ohlcv.plot(plot_type='candlestick')


FigureWidget({
    'data': [{'close': array([ 5.131,  5.038,  4.781, ..., 22.64 , 22.97 , 24.79 ]),
              'decreasing': {'line': {'color': '#d95f02'}},
              'high': array([ 5.131,  5.21 ,  5.1  , ..., 22.77 , 23.08 , 24.88 ]),
              'increasing': {'line': {'color': '#1b9e76'}},
              'low': array([ 5.023,  5.012,  4.693, ..., 21.79 , 21.7  , 22.58 ]),
              'name': 'Candlestick',
              'open': array([ 5.039,  5.141,  5.078, ..., 21.99 , 22.04 , 22.74 ]),
              'type': 'candlestick',
              'uid': '38799ca7-26c2-4882-b7d0-b6b9ef72a256',
              'x': array([datetime.datetime(2015, 12, 1, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 2, 5, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2015, 12, 3, 5, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2023, 3, 14, 4, 0, tzinfo=datetime.timezone.utc),
             

In [110]:
def pullback_place_entry(start, close, rsi_window = 10, sma_window = 200, entry = 30):
    rsi = vbt.RSI.run(close, window=rsi_window).rsi.to_numpy()
    sma = vbt.MA.run(close, window=sma_window).ma.to_numpy()

    entry = np.where((rsi < entry) & (sma < start), 1, 0)

    entry = np.roll(entry, 2)

    return entry, rsi, sma

PULLBACK_PLACE_ENTRY = vbt.IndicatorFactory(
    class_name = "pullback_place_order",
    short_name = "pullback_place_order",
    input_names = ['start', 'close'],
    output_names = ['value', 'rsi', 'sma'],
    param_names=['rsi_window', 'sma_window', 'entry']
).from_apply_func(pullback_place_entry, rsi_window = 10, sma_window = 200, entry = 30)

enter = PULLBACK_PLACE_ENTRY.run(
    data['Open'],
    data['Close'],
    rsi_window = np.arange(10,30,step=1,dtype=int),
    sma_window = np.arange(50,300,step=5,dtype=int),
    #entry = np.arange(10,50,step=2,dtype=int),
    param_product = True
)

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    display(data)

,Open,High,Low,Close,Volume
timestamp,,,,,
2015-12-01 05:00:00+00:00,5.039,5.131,5.023,5.131,58509312.0
2015-12-02 05:00:00+00:00,5.141,5.210,5.012,5.038,61046808.0
2015-12-03 05:00:00+00:00,5.078,5.100,4.693,4.781,112956096.0
2015-12-04 05:00:00+00:00,4.822,5.139,4.793,5.109,86644920.0
2015-12-07 05:00:00+00:00,5.125,5.134,4.972,5.047,69225576.0
2015-12-08 05:00:00+00:00,4.887,5.094,4.881,5.047,81298944.0
2015-12-09 05:00:00+00:00,4.976,5.061,4.726,4.812,144867168.0
2015-12-10 05:00:00+00:00,4.847,4.976,4.805,4.880,81462864.0
2015-12-11 05:00:00+00:00,4.711,4.749,4.535,4.546,147436104.0


In [ ]:
def pullback_place_exit(enter, rsi, out = 40, leave = 10):
    leave_array = np.roll(enter.astype(float), leave)
    leave_array[:leave] = np.inf
    #print(leave_array)

    stop_loss =  np.where((leave_array == 1), -1, 0)
    take_profit = np.where(rsi>out, -1, 0)
    sell = np.where((leave_array == 1) | (rsi>out) , -1, 0)

    sell = np.roll(sell, 2)

    return take_profit, stop_loss, sell

PULLBACK_PLACE_EXIT = vbt.IndicatorFactory(
    class_name = "pullback_place_order",
    short_name = "pullback_place_order",
    input_names = ['enter','rsi'],
    output_names = ['take_profit', 'stop_loss', 'sell'],
    param_names=['out', 'leave']
).from_apply_func(pullback_place_exit, out = 40, leave=10)

exit = PULLBACK_PLACE_EXIT.run(
    enter.value,
    enter.rsi,
    out= np.arange(10,70,step=2,dtype=int),
    #leave=np.arange(5,15,step=1,dtype=int),
    param_product = True
)

with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       ):
    display(data)


exits = exit.sell == -1
entries = enter.value == 1

pf = vbt.Portfolio.from_signals(data['Open'], entries, exits, init_cash=10000, freq='1d')

#orders = pf.orders

returns = pf.total_return()
print(returns)
print(returns.max())
'''
fig = returns.vbt.volume(
        x_level = "pullback_place_order_rsi_window",
        y_level = "pullback_place_order_sma_window",
        z_level = "pullback_place_order_out",
        trace_kwargs=dict(colorbar=dict(title='Total return', tickformat='%'))
)
fig.show()'''